# ResNet50 with Data Augmentation 

In [1]:
#Res_Net 50 
#All New RGB images - 17 participant 
# Data Augmented
# Res-Net50 modified 

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Activation
from keras.layers import Input
from keras.utils import multi_gpu_model
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input
from keras.layers import Flatten,Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image 
from keras.models import Model
from keras.applications.vgg16 import VGG16, preprocess_input



Using TensorFlow backend.


#### Loading data 

In [3]:
x_train = np.load('/cai_lab/thomas/DWD/Numpy_RGB/train/x_data_all.npy')    # data path 
y_train = np.load('/cai_lab/thomas/DWD/Numpy_RGB/train/y_data_all.npy')

x_valid = np.load('/cai_lab/thomas/DWD/Numpy_RGB/valid/x_data_all.npy')
y_valid = np.load('/cai_lab/thomas/DWD/Numpy_RGB/valid/y_data_all.npy')

print('x_train.shape:',x_train.shape,'y_train.shape:',y_train.shape)
print('x_valid.shape:',x_valid.shape,'y_valid.shape:',y_valid.shape)

x_train.shape: (17194, 224, 224, 3) y_train.shape: (17194, 9)
x_valid.shape: (2558, 224, 224, 3) y_valid.shape: (2558, 9)


In [4]:
batch_size = 64
resize = 224
n_class = 9

## Model (ResNet 50)

In [5]:
## Defining the input

from keras.layers import Input
resnet50_input = Input(shape = (224, 224, 3), name = 'Image_input')



from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.applications.resnet50 import ResNet50



model_resnet50_conv = ResNet50(weights= 'imagenet', include_top=False, input_shape= (224,224,3))
model_resnet50_conv.summary()

W0420 06:02:56.560369 139797889578752 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [6]:

output_resnet50_conv = model_resnet50_conv(resnet50_input)

#Adding fully-connected layers 

y=GlobalAveragePooling2D()(output_resnet50_conv)
y=Dense(1024,activation='relu')(y) #we add dense layers so that the model can learn more complex functions and classify for better results.
y = Dropout(0.5)(y) 
y=Dense(1024,activation='relu')(y) 
y = BatchNormalization()(y)
y = Dropout(0.5)(y)
y=Dense(512,activation='relu')(y) 
y = Dense(n_class, activation='softmax', name='predictions')(y)


resnet50_pretrained = Model(input = resnet50_input, output = y)



resnet50_pretrained.summary()




Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Image_input (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096

## Model Saving and Compling 

In [7]:

save_name = 'ResNet50_RGB_with_DataAug'
save_path = './checkpoints/' + save_name

if os.path.exists(save_path)==False:
    os.makedirs(save_path)

checkpoint = ModelCheckpoint(save_path +'/checkpoint-{epoch:04d}.h5', verbose=1, monitor='val_loss', save_best_only=False, mode='auto', period=1)
tensorboard = TensorBoard(log_dir='./graph/'+ save_name, histogram_freq=0, write_graph=True, write_images=True)
callback_list = [checkpoint, tensorboard]

resnet50_pretrained = multi_gpu_model(resnet50_pretrained, gpus=2)

aug = ImageDataGenerator(
        rotation_range=20,
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        horizontal_flip=True,
        fill_mode="nearest")


# Compiling


adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0)
sgd = optimizers.SGD(lr = 0.001)
resnet50_pretrained.compile(loss='categorical_crossentropy',optimizer = sgd,metrics=['accuracy'])

    
resnet50_pretrained.fit_generator(aug.flow(x_train, y_train, batch_size=batch_size),
                    validation_data=(x_valid, y_valid),
                    steps_per_epoch = len(x_train) // batch_size,        
                    epochs=100,
                    verbose=1,
                    callbacks=callback_list)


resnet50_pretrained.save(save_path + '/epoch-finish.h5')
resnet50_pretrained.save_weights(save_path +'/epoch-finish_weights.h5')

W0420 06:03:20.643270 139797889578752 deprecation.py:323] From /home/cai/.local/lib/python3.5/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0420 06:03:32.289013 139797889578752 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W0420 06:03:32.311185 139797889578752 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras/callbacks/tensorboard_v1.py:200: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0420 06:03:32.311836 139797889578752 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras

Epoch 1/100
268/268 [==============================] - 225s 839ms/step - loss: 1.9486 - accuracy: 0.3423 - val_loss: 1.2707 - val_accuracy: 0.5657

Epoch 00001: saving model to ./checkpoints/ResNet50_RGB_with_DataAug/checkpoint-0001.h5


W0420 06:07:44.207581 139797889578752 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras/callbacks/tensorboard_v1.py:343: The name tf.Summary is deprecated. Please use tf.compat.v1.Summary instead.



Epoch 2/100
268/268 [==============================] - 214s 800ms/step - loss: 0.9761 - accuracy: 0.6756 - val_loss: 0.8889 - val_accuracy: 0.7017

Epoch 00002: saving model to ./checkpoints/ResNet50_RGB_with_DataAug/checkpoint-0002.h5
Epoch 3/100
268/268 [==============================] - 214s 799ms/step - loss: 0.6071 - accuracy: 0.8122 - val_loss: 0.7894 - val_accuracy: 0.7647

Epoch 00003: saving model to ./checkpoints/ResNet50_RGB_with_DataAug/checkpoint-0003.h5
Epoch 4/100
268/268 [==============================] - 213s 794ms/step - loss: 0.4429 - accuracy: 0.8678 - val_loss: 0.7715 - val_accuracy: 0.7690

Epoch 00004: saving model to ./checkpoints/ResNet50_RGB_with_DataAug/checkpoint-0004.h5
Epoch 5/100
268/268 [==============================] - 213s 794ms/step - loss: 0.3551 - accuracy: 0.8942 - val_loss: 0.7287 - val_accuracy: 0.7819

Epoch 00005: saving model to ./checkpoints/ResNet50_RGB_with_DataAug/checkpoint-0005.h5
Epoch 6/100
268/268 [==============================] - 2


Epoch 00036: saving model to ./checkpoints/ResNet50_RGB_with_DataAug/checkpoint-0036.h5
Epoch 37/100
268/268 [==============================] - 206s 770ms/step - loss: 0.0524 - accuracy: 0.9832 - val_loss: 0.8350 - val_accuracy: 0.7885

Epoch 00037: saving model to ./checkpoints/ResNet50_RGB_with_DataAug/checkpoint-0037.h5
Epoch 38/100
268/268 [==============================] - 206s 769ms/step - loss: 0.0501 - accuracy: 0.9846 - val_loss: 0.8745 - val_accuracy: 0.7838

Epoch 00038: saving model to ./checkpoints/ResNet50_RGB_with_DataAug/checkpoint-0038.h5
Epoch 39/100
268/268 [==============================] - 206s 768ms/step - loss: 0.0497 - accuracy: 0.9841 - val_loss: 0.8620 - val_accuracy: 0.7877

Epoch 00039: saving model to ./checkpoints/ResNet50_RGB_with_DataAug/checkpoint-0039.h5
Epoch 40/100
268/268 [==============================] - 206s 769ms/step - loss: 0.0489 - accuracy: 0.9855 - val_loss: 0.8724 - val_accuracy: 0.7826

Epoch 00040: saving model to ./checkpoints/ResNet50